# Import library

In [3]:
import sys
sys.path.append('src')
import en_system_ex_analysis as enex
from SALib.sample import saltelli
from SALib.analyze import sobol
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dartwork_mpl as dm
import seaborn as sns
from scipy.stats import norm, uniform, triang, gamma
import warnings
from pprint import pprint
warnings.filterwarnings("ignore")

Load colors...
Load colormaps...


# Primary energy use, CO2 emission, Exergy efficiency

## system setting

In [3]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s

EB       = enex.ElectricBoiler()
EB.r0    = 0.2
EB.H     = 0.8
EB.x_ins = 0.05
EB.dV_w_tap = MWF
EB.system_update()

GB = enex.GasBoiler()
GB.r0    = 0.12
GB.H     = 0.45
GB.x_ins = 0.05
GB.dV_w_tap = MWF
GB.system_update()

HPB = enex.HeatPumpBoiler()
HPB.r0    = 0.2
HPB.H     = 0.8
HPB.x_ins = 0.05
HPB.dV_w_tap = MWF
HPB.system_update()

print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')
print(f'Volume of water in the tank:')

Mean water flow rate: 1.11 L/min
Volume of water in the tank:


## Plot

In [8]:

# Primary energy factor
PEF_elec = 2.75 
PEF_NG = 1.1

# CO2 emission factor
Wh2MJ = 10**6/(1*enex.h2s) # enex.h2s = 3600 s/h
C_to_CO2 = 44/12 # 탄소 배출량당 이산화탄소 배출량 분자량에 의한 환산계수 
CF_NG_ori = 15.281 # tC/TJ = kgC/GJ = gC/MJ

CF_elec = 0.4747 # tCO2/MWh = gCO2/Wh
CF_NG = (CF_NG_ori*C_to_CO2)/Wh2MJ # gCO2/Wh

# Energy use
energy_use = [
        EB.E_heater,
        GB.E_NG,
        HPB.E_cmp + HPB.E_fan
]

# Exergy use
exergy_use = [
        EB.X_heater,
        GB.X_NG,
        HPB.X_cmp + HPB.X_fan
]

# Primary energy use
primary_energy_use = [
        energy_use[0] * PEF_elec,
        energy_use[1] * PEF_NG,
        energy_use[2] * PEF_elec,
]

# CO2 emission
CO2_emission = [
        energy_use[0] * CF_elec, 
        energy_use[1] * CF_NG,
        energy_use[2] * CF_elec
]

eX_efficiency = np.array([
        EB.X_eff, GB.X_eff, HPB.X_eff
]) * 100

# Create figure and axis - adjust figure size for vertical layout
nrows = 1
ncols = 4
fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(16), dm.cm2in(3.5)), dpi=200, facecolor=None, edgecolor='k')

for ridx in range(nrows):
    for cidx in range(ncols):
        idx    = ridx * ncols + cidx
        data   = [energy_use, primary_energy_use, CO2_emission, eX_efficiency][idx]
        heading   = ['Energy use [W]', 'Primary energy use [W]', 'CO$_2$ emission [gCO$_2$/h]', 'Exergy efficiency [%]'][idx]
        labels = [['EB', 'GB', 'HPB'], # ['EB\n(E_heater)', 'GB\n(E_NG})', 'HPB\n(E_fan} + E_cmp})']
                  ['EB', 'GB', 'HPB'],
                  ['EB', 'GB', 'HPB'],
                  ['EB', 'GB', 'HPB']][idx]

        group_margin = 0.15  # margin between groups
        bar_margin   = 0.1  # margin between bars
        bar_width    = 0.25

        dN     = len(data)  # number of data
        x_pos0 = np.array([i * (bar_margin + bar_width) for i in range(dN)]) # 바 margin과 bar width를 곱해서 그룹 마진이 없는 x_pos0에 저장
        gl1    = [[1,1,1],[1,1,1],[1,1,1],[1,1,1]][idx]  # group
        gN     = len(gl1)  # number of groups
        gl2    = np.cumsum(gl1)

        x_pos = x_pos0[:gl2[0]].tolist()
        for i in range(gN - 1):  # Add group_margin between arrays
            x_pos.extend((x_pos0[gl2[i]:gl2[i+1]] + group_margin * (i + 1)).tolist()) # 그룹 마진을 추가

        # Plot parameters
        sp_ratio = 0.5
        xmar = bar_width
        xmax = max(x_pos)
        xmin = min(x_pos)
        ymar = 0
        ymax = [9000, 9000, 1500, 30][idx]
        ymin = [0, 0, 0, 0][idx]
        yint = [3000, 3000, 500, 10][idx]

        # Create vertical bars
        bars = ax[idx].bar(x_pos, data, bar_width, color=[['dm.red7', 'dm.red5','dm.red3'],
                                                          ['dm.yellow7', 'dm.yellow5','dm.yellow3'],
                                                          ['dm.lime7', 'dm.lime5','dm.lime3'],
                                                          ['dm.grape7', 'dm.grape5','dm.grape3']][idx])

        # Annotation
        ax[idx].annotate(f'{heading}', xy=(0.5, 1.1), xycoords='axes fraction', color='dm.gray9',
                        horizontalalignment='center', verticalalignment='bottom', fontsize=dm.fs(0), fontweight=400)

        # Add value labels on top of each bar
        for bidx, bar in enumerate(bars):
                height = bar.get_height()
                dc = [0,0,0,1][cidx]
                ax[idx].text(bar.get_x() + bar.get_width() / 2., height + ymax / 150,
                                f'{height:.{dc}f}', ha='center', va='bottom', fontsize=dm.fs(-2.5),fontweight=300)

            
        # Axes limits
        ax[idx].set_xlim(xmin - xmar, xmax + xmar)
        ax[idx].set_ylim(ymin - ymar, ymax + ymar)


        # Tick parameters
        ax[idx].tick_params(axis='x', direction='in', labelsize=dm.fs(-1.5),
                            which='major', length=0, width=0.5 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='x', direction='in', labelsize=dm.fs(-1.5),
                            which='minor', length=0, width=0.25 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='y', direction='out', labelsize=dm.fs(-1.5),
                            which='major', length=2, width=0.5 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='y', direction='out', labelsize=dm.fs(-1.5),
                            which='minor', length=1, width=0.25 * sp_ratio, pad=3)

        # Labels and ticks
        ax[idx].set_xticks(x_pos)
        ax[idx].set_yticks(np.arange(ymin,ymax+yint,yint))  # No y-ticks
        ax[idx].set_xticklabels(labels, ha='center', fontsize=dm.fs(-1.5), fontweight=400)
        ax[idx].set_yticklabels(np.arange(ymin,ymax+yint,yint), fontsize=dm.fs(-1.5), fontweight=300)
        
        # Minor ticks
        # ax[idx].xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
        # ax[idx].yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

        # Spines
        for k in ['top','right']:  # Only keep bottom spine
            ax[idx].spines[k].set_visible(False)
        ax[0].spines['bottom'].set_linewidth(0.5 * sp_ratio)
        ax[0].spines['bottom'].set_color('k')

plt.subplots_adjust(wspace=0.3)
dm.simple_layout(fig, bbox=(0.01,1,0.0,.945), verbose=False)
img_stem = f'figure/fig7'
plt.savefig(f'{img_stem}.svg')
plt.savefig(f'{img_stem}.png',dpi=600)
dm.util.save_and_show(fig)


# Monte Carlo simulation

## EB

In [ ]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
problem = {
    'num_vars': 3,
    'names'   : ['Insulation thick [m]', 'Outdoor air temp [°C]', "Supply water temp [°C]"],
    'bounds'  : [
        [0.01, 0.1]                  ,    # x_ins: 단열 두께 (m)
        [enex.C2K(-10), enex.C2K(10)] ,            # T_oa: 실외 온도 (°C)
        [enex.C2K(4), enex.C2K(10)]   ,            # T_sw: 상수도 온도 (°C)
    ],
    'dists': ['unif', 'unif', 'unif']  # 분포 형상 (균등, 삼각, 정규)
}

# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations)

# 엑서지 효율 저장 리스트
ex_eff_EB = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):  
    # enex.ElectricBoiler 객체 생성
    boiler = enex.ElectricBoiler()

    # 확률 변수에서 추출한 변수 값들 할당
    boiler.x_ins              = param_values[i, 0]  
    boiler.T0                 = param_values[i, 1] 
    boiler.T_w_sup            = param_values[i, 2] 

    # 시스템 업데이트 수행
    boiler.system_update()

    # 엑서지 효율 계산
    X_w_serv = boiler.exergy_balance["mixing valve"]["out"]["X_w_serv"]
    E_heater = boiler.exergy_balance["hot water tank"]["in"]["E_heater"]

    eta_exergy = X_w_serv / E_heater
    # print(eta_exergy)
    ex_eff_EB.append(eta_exergy)
    
    if eta_exergy < 0:
        print(f"h_co: {boiler.h_co}, eta_exergy: {eta_exergy}")

# 데이터 시각화
nrows = 1
ncols = 1

xmin, xmax, xint, xmar = 0, 15, 3, 0
ymin, ymax, yint, ymar = 0, 0.2, 0.05,  0

fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)
for ridx in range(nrows): 
    for cidx in range(ncols): 
        idx = ridx * ncols + cidx

        # Plot histogram
        sns.kdeplot(np.array(ex_eff_EB) * 100, ax=ax, color='tw.indigo:700', fill=True, linewidth=0.75)

        # tick limits 
        ax.set_xlim(xmin - xmar, xmax + xmar)
        ax.set_ylim(ymin - ymar, ymax + ymar)
        
        # set ticks
        ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
        ax.set_yticks(np.arange(ymin, ymax + yint, yint))
        
        # print x, y range
        # print(f"y max: {max_frequency}")
        
# set tick parameters
plt.xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
plt.ylabel("Probability density", fontsize=dm.fs(0), fontweight=300)

# layout
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# save
fig_name = 'ex_eff_distribution_EB'
folder_path = r'figure'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)

# show
dm.util.save_and_show(fig)
plt.close()

# print
print(f"Exergy efficiency range: {min(ex_eff_EB)*100:.2f} ~ {max(ex_eff_EB)*100:.2f}")
print(f"\n\n{'='*10} Problem setting {'='*10}\n")
pprint(problem)

Exergy efficiency range: 4.89 ~ 14.36


========== Problem setting ==========

{'bounds': [[0.01, 0.1], [263.15, 283.15], [277.15, 283.15]],
 'dists': ['unif', 'unif', 'unif'],
 'names': ['Insulation thick [m]',
           'Outdoor air temp [°C]',
           'Supply water temp [°C]'],
 'num_vars': 3,
 'sample_scaled': True}


## GB

In [ ]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
problem = {
    'num_vars': 4,
    'names'   : ['Insulation thick [m]', 'Outdoor air temp [°C]', "Supply water temp [°C]", "boiler eff [-]"],
    'bounds'  : [
        [0.01, 0.1],    # x_ins      : 단열 두께 (m)
        [enex.C2K(-10), enex.C2K(10)],    # T_oa: 실외 온도 (°C)
        [enex.C2K(4), enex.C2K(10)],    # T_sw: 상수도 온도 (°C)
        [0.8, 0.9],    # COP: COP
    ],
    'dists': ['unif', 'unif', 'unif', 'unif']  
}

# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations)

# 엑서지 효율 저장 리스트
ex_eff_GB = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):  
    # enex.ElectricBoiler 객체 생성
    boiler = enex.GasBoiler()

    # 확률 변수에서 추출한 변수 값들 할당
    boiler.x_ins              = param_values[i, 0]  # 단열 두께
    boiler.T0                 = param_values[i, 1]  # 대류 열전달 계수
    boiler.T_w_sup            = param_values[i, 2]  # 상수도 온도
    boiler.eta_boiler         = param_values[i, 3]  # 보일러 효율

    # 시스템 업데이트 수행
    boiler.system_update()

    # 엑서지 효율 계산
    X_w_serv = boiler.exergy_balance["mixing valve"]["out"]["X_w,serv"]
    X_NG = boiler.exergy_balance["combustion chamber"]["in"]["X_NG"]
    
    eta_exergy = X_w_serv / X_NG
    ex_eff_GB.append(eta_exergy)

# 데이터 시각화
nrows = 1
ncols = 1

xmin, xmax, xint, xmar = 0, 15, 3, 0
ymin, ymax, yint, ymar = 0, 0.2,  0.05,  0

fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)
for ridx in range(nrows): 
    for cidx in range(ncols): 
        idx = ridx * ncols + cidx

        # Plot histogram
        sns.kdeplot(np.array(ex_eff_GB) * 100, ax=ax, color='tw.indigo:700', fill=True, linewidth=0.75)

        # tick limits 
        ax.set_xlim(xmin - xmar, xmax + xmar)
        ax.set_ylim(ymin - ymar, ymax + ymar)
        
        # set ticks
        ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
        ax.set_yticks(np.arange(ymin, ymax + yint, yint))
        
        # print x, y range
        # print(f"y max: {max_frequency}")

# set tick parameters
plt.xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
plt.ylabel("Probability density", fontsize=dm.fs(0), fontweight=300)

# layout
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# save
fig_name = 'ex_eff_distribution_GB'
folder_path = r'figure'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)

# show
dm.util.save_and_show(fig)
plt.close()

# print
print(f"Exergy efficiency range: {min(ex_eff_GB)*100:.2f} ~ {max(ex_eff_GB)*100:.2f}")
print(f"\n\n{'='*10} Problem setting {'='*10}\n")
pprint(problem)

Exergy efficiency range: 4.23 ~ 13.73


========== Problem setting ==========

{'bounds': [[0.01, 0.1], [263.15, 283.15], [277.15, 283.15], [0.8, 0.9]],
 'dists': ['unif', 'unif', 'unif', 'unif'],
 'names': ['Insulation thick [m]',
           'Outdoor air temp [°C]',
           'Supply water temp [°C]',
           'boiler eff [-]'],
 'num_vars': 4,
 'sample_scaled': True}


## HPB

In [ ]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
problem = {
    'num_vars': 4,
    'names'   : ['Insulation thick [m]',  'Outdoor air temp [°C]', "Supply water temp [°C]", "COP [-]"],
    'bounds'  : [
        [0.01, 0.1],    # x_ins      : 단열 두께 (m)
        [enex.C2K(-10), enex.C2K(10)],    # T_oa: 실외 온도 (°C)
        [enex.C2K(4), enex.C2K(10)],    # T_sw: 상수도 온도 (°C)
        [1.5, 2.5],    # COP: COP
    ],
    'dists': ['unif', 'unif', 'unif', 'unif']  # 분포 형상 (균등, 삼각, 정규)
}

# Saltelli 방법을 사용하여 샘플 생성 -> 최종 샘플 개수는 num_simulations * (num_vars + 2) = 10000 * (5 + 2) = 70000
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_HPB = []
cop_list = []
T_r_ext_list = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):  
    # enex.ElectricBoiler 객체 생성
    boiler = enex.HeatPumpBoiler()

    # 확률 변수에서 추출한 변수 값들 할당
    boiler.x_ins              = param_values[i, 0]  # 단열 두께
    boiler.T0                 = param_values[i, 1]  # 대류 열전달 계수
    boiler.T_w_sup            = param_values[i, 2]  # 상수도 온도
    
    # COP calculation
    dT = boiler.T_w_tank - boiler.T0 # 온수 탱크 온도와 실외 온도 차이
    boiler.COP_hp             = 5.06-0.05*dT +0.00006*(dT**2) # https://www.notion.so/betlab/Model-based-flexibility-assessment-of-a-residential-heat-pump-pool-1af6947d125d8037a108e85bb6985dee?pvs=4
    cop_list.append(boiler.COP_hp)
    

    # 시스템 업데이트 수행
    boiler.system_update()
    T_r_ext_list.append(boiler.T_r_ext)

    # 엑서지 효율 계산
    X_w_serv = boiler.exergy_balance["mixing valve"]["out"]["X_w,serv"]
    E_fan    = boiler.exergy_balance["external unit"]["in"]["E_fan"]
    E_cmp    = boiler.exergy_balance["refrigerant loop"]["in"]["E_cmp"]
    
    eta_exergy = X_w_serv / (E_fan + E_cmp)
    ex_eff_HPB.append(eta_exergy)

# 데이터 시각화
nrows = 1
ncols = 1

xmin, xmax, xint, xmar = 0, 40, 10, 0
ymin, ymax, yint, ymar = 0, 0.15,  0.05,  0

fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)
for ridx in range(nrows): 
    for cidx in range(ncols): 
        idx = ridx * ncols + cidx

        # Plot histogram
        sns.kdeplot(np.array(ex_eff_HPB) * 100, ax=ax, color='tw.indigo:700', fill=True, linewidth=0.75)

        # tick limits 
        ax.set_xlim(xmin - xmar, xmax + xmar)
        ax.set_ylim(ymin - ymar, ymax + ymar)
        
        # set ticks
        ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
        ax.set_yticks(np.arange(ymin, ymax + yint, yint))
        
        # print x, y range
        print(f"Exergy efficiency range: {min(ex_eff_HPB)*100:.2f} ~ {max(ex_eff_HPB)*100:.2f}")
        # print(f"y max: {max_frequency}")
        

# set tick parameters
plt.xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
plt.ylabel("Probability density", fontsize=dm.fs(0), fontweight=300)

# layout
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# save
fig_name = 'ex_eff_distribution_HPB'
folder_path = r'figure'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)

# show
dm.util.save_and_show(fig)
plt.close()

# print
print(f"Exergy efficiency range: {min(ex_eff_HPB)*100:.2f} ~ {max(ex_eff_HPB)*100:.2f}")
print(f"cop range: {min(cop_list):.2f} ~ {max(cop_list):.2f}")
print(f"\n\n{'='*10} Problem setting {'='*10}\n")
pprint(problem)

# # sobol analysis
# Si = sobol.analyze(problem, np.array(ex_eff_HPB), calc_second_order=True, print_to_console=True)

# # 1차 민감도 지수 출력
# print("\n1차 민감도 지수 (S1):")
# for name, s1 in zip(problem["names"], Si["S1"]):
#     print(f"{name}: {s1:.4f}")

Exergy efficiency range: 11.75 ~ 25.23


T_r_ext: -19.9993896484375, -0.0006103515625
Exergy efficiency range: 11.75 ~ 25.23
cop range: 1.85 ~ 2.71


========== Problem setting ==========

{'bounds': [[0.01, 0.1], [263.15, 283.15], [277.15, 283.15], [1.5, 2.5]],
 'dists': ['unif', 'unif', 'unif', 'unif'],
 'names': ['Insulation thick [m]',
           'Outdoor air temp [°C]',
           'Supply water temp [°C]',
           'COP [-]'],
 'num_vars': 4,
 'sample_scaled': True}


## EB, GB, HPB
- EB, GB, HPB 코드를 모두 실행하여야 돌아감

In [ ]:
# 데이터 시각화
nrows = 1
ncols = 1

xmin, xmax, xint, xmar = 0, 40, 10, 0
ymin, ymax, yint, ymar = 0, 0.2,  0.05,  0

fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(14), dm.cm2in(7)), dpi=600)


# Plot histogram
sns.kdeplot(np.array(ex_eff_EB) * 100, ax=ax, color='tw.orange:700', fill=True, linewidth=0.75)
sns.kdeplot(np.array(ex_eff_GB) * 100, ax=ax, color='tw.pink:700', fill=True, linewidth=0.75)
sns.kdeplot(np.array(ex_eff_HPB) * 100, ax=ax, color='tw.indigo:700', fill=True, linewidth=0.75)

# tick limits 
ax.set_xlim(xmin - xmar, xmax + xmar)
ax.set_ylim(ymin - ymar, ymax + ymar)

# tick parameters
ax.tick_params(axis='x', labelsize=dm.fs(1), which='major')
ax.tick_params(axis='y', labelsize=dm.fs(1), which='major')

# set ticks
ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
ax.set_yticks(np.arange(ymin, ymax + yint, yint))

# legend 
ax.legend(['Electric boiler', 'Gas boiler', 'Heat pump boiler'], fontsize=dm.fs(0),)

# print x, y range
print(f"Exergy efficiency range: {min(ex_eff_HPB)*100:.2f} ~ {max(ex_eff_HPB)*100:.2f}")
# print(f"y max: {max_frequency}")


plt.xlabel("Exergy efficiency [%]", fontsize=dm.fs(2), fontweight=300 )
plt.ylabel("Probability density", fontsize=dm.fs(2), fontweight=300)
# plt.title("Exergy Efficiency Distribution from Monte Carlo Simulation", fontsize=dm.fs(2), fontweight=400)
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)
fig_name = 'ex_eff_distribution_all'
folder_path = r'figure'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)
dm.util.save_and_show(fig)
plt.close()

Exergy efficiency range: 11.75 ~ 25.23


## ASHP

In [ ]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
problem = {
    'num_vars': 4,
    'names'   : ['Insulation thick [m]',  'Outdoor air temp [°C]', "Supply water temp [°C]", "COP [-]"],
    'bounds'  : [
        [0.01, 0.1],    # x_ins      : 단열 두께 (m)
        [enex.C2K(-10), enex.C2K(10)],    # T_oa: 실외 온도 (°C)
        [enex.C2K(4), enex.C2K(10)],    # T_sw: 상수도 온도 (°C)
        [1.5, 2.5],    # COP: COP
    ],
    'dists': ['unif', 'unif', 'unif', 'unif']  # 분포 형상 (균등, 삼각, 정규)
}

# Saltelli 방법을 사용하여 샘플 생성 -> 최종 샘플 개수는 num_simulations * (num_vars + 2) = 10000 * (5 + 2) = 70000
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_HPB = []
cop_list = []
T_r_ext_list = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):  
    # enex.ElectricASHP 객체 생성
    ASHP = enex.HeatPumpASHP()

    # 확률 변수에서 추출한 변수 값들 할당
    ASHP.x_ins              = param_values[i, 0]  # 단열 두께
    ASHP.T0                 = param_values[i, 1]  # 대류 열전달 계수
    ASHP.T_w_sup            = param_values[i, 2]  # 상수도 온도
    
    # COP calculation
    dT = ASHP.T_w_tank - ASHP.T0 # 온수 탱크 온도와 실외 온도 차이
    ASHP.COP_hp             = 5.06-0.05*dT +0.00006*(dT**2) # https://www.notion.so/betlab/Model-based-flexibility-assessment-of-a-residential-heat-pump-pool-1af6947d125d8037a108e85bb6985dee?pvs=4
    
    
    cop_list.append(ASHP.COP_hp)
    

    # 시스템 업데이트 수행
    ASHP.system_update()
    T_r_ext_list.append(ASHP.T_r_ext)

    # 엑서지 효율 계산
    X_w_serv = ASHP.exergy_balance["mixing valve"]["out"]["X_w,serv"]
    E_fan    = ASHP.exergy_balance["external unit"]["in"]["E_fan"]
    E_cmp    = ASHP.exergy_balance["refrigerant loop"]["in"]["E_cmp"]
    
    eta_exergy = X_w_serv / (E_fan + E_cmp)
    ex_eff_HPB.append(eta_exergy)

# 데이터 시각화
nrows = 1
ncols = 1

xmin, xmax, xint, xmar = 0, 40, 10, 0
ymin, ymax, yint, ymar = 0, 0.15,  0.05,  0

fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)
for ridx in range(nrows): 
    for cidx in range(ncols): 
        idx = ridx * ncols + cidx

        # Plot histogram
        sns.kdeplot(np.array(ex_eff_HPB) * 100, ax=ax, color='tw.indigo:700', fill=True, linewidth=0.75)

        # tick limits 
        ax.set_xlim(xmin - xmar, xmax + xmar)
        ax.set_ylim(ymin - ymar, ymax + ymar)
        
        # set ticks
        ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
        ax.set_yticks(np.arange(ymin, ymax + yint, yint))
        
        # print x, y range
        print(f"Exergy efficiency range: {min(ex_eff_HPB)*100:.2f} ~ {max(ex_eff_HPB)*100:.2f}")
        # print(f"y max: {max_frequency}")
        

# set tick parameters
plt.xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
plt.ylabel("Probability density", fontsize=dm.fs(0), fontweight=300)

# layout
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# save
fig_name = 'ex_eff_distribution_HPB'
folder_path = r'figure'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)

# show
dm.util.save_and_show(fig)
plt.close()

# print
print(f"Exergy efficiency range: {min(ex_eff_HPB)*100:.2f} ~ {max(ex_eff_HPB)*100:.2f}")
print(f"cop range: {min(cop_list):.2f} ~ {max(cop_list):.2f}")
print(f"\n\n{'='*10} Problem setting {'='*10}\n")

In [ ]:
# Sobol sensitivity analysis
Si = sobol.analyze(problem, np.array(ex_eff_EB), print_to_console=False)

# Visualize first-order sensitivity indices
fig, ax = plt.subplots(figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)

# Bar plot for first-order sensitivity indices
ax.bar(problem['names'], Si['S1'], color='tw.indigo:700', alpha=0.7)

# Add labels and title
ax.set_ylabel('First-order Sensitivity Index', fontsize=dm.fs(0), fontweight=300)
ax.set_xlabel('Input Variables', fontsize=dm.fs(0), fontweight=300)
ax.set_title('Sobol Sensitivity Analysis', fontsize=dm.fs(1), fontweight=400)

# Layout adjustments
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# Save and show the figure
fig_name = 'sobol_sensitivity_EB'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)
dm.util.save_and_show(fig)
plt.close()

# Water fall chart

## EB

In [9]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')

EB       = enex.ElectricBoiler()
EB.k_shell = 25
EB.r0    = 0.2
EB.H     = 0.8
EB.x_ins = 0.05
EB.dV_w_serv = MWF
EB.system_update()

# ----------------------------
# 데이터 정의 (Energy, Entropy, Exergy 각각 Tank + Mixing 포함)
# ----------------------------
# 데이터 정의 및 딕셔너리 구성
Energy = {
    "components": [
        {"value": EB.Q_w_sup_tank, "sign": 1, "label": 'Supply water\nto tank'},
        {"value": EB.E_heater, "sign": 1, "label": 'Electricity input\nto heater'},
        {"value": 0, "sign":  1, "label": ''},
        {"value": EB.Q_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": EB.Q_w_sup_mix, "sign":  1, "label": 'Supply water\nto mixing valve'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": EB.Q_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax"   : 3600,
    "yint"   : 900
}
    
Entropy = {
    "components": [
        {"value": EB.S_w_sup_tank, "sign": 1, "label": 'Supply water\nto tank'},
        {"value": EB.S_heater, "sign": 1, "label": 'Electricity input\nto heater'},
        {"value": EB.S_g_tank, "sign": 1, "label": 'Consumption\nin tank'},
        {"value": EB.S_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": EB.S_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixing valve'},
        {"value": EB.S_g_mix, "sign": 1, "label": 'generated (mix)'},
        {"value": EB.S_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.amber:',
    "ylabel": 'Entropy [W/K]',
    "ymax"   : 16,
    "yint"   : 4
}
    
Exergy = {
    "components": [
        {"value": EB.X_w_sup_tank, "sign": 1, "label": 'Supply water\nto tank'},
        {"value": EB.X_heater, "sign": 1, "label": 'Electricity input\nto heater'},
        {"value": EB.X_c_tank, "sign": -1, "label": 'Consumption\nin tank'},
        {"value": EB.X_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": EB.X_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixing valve'},
        {"value": EB.X_c_mix, "sign": -1, "label": 'Consumption\nin mixing valve'},
        {"value": EB.X_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.purple:',
    "ylabel":  'Exergy [W]',
    "ymax"   : 3600,
    "yint"   : 900
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 3
ncols = 1

fig, axs = plt.subplots(2, 1, figsize=(dm.cm2in(14), dm.cm2in(8)), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.05  
    bm = DL * 0.06 
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-1))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(min(0, min(value)*1.1), data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(0), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.065, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/EB_waterfall.svg')
plt.savefig('figure/EB_waterfall.png', dpi=600)
dm.util.save_and_show(fig)

Mean water flow rate: 1.11 L/min


In [50]:
EB.Q_l_tank/EB.Q_w_serv
EB.X_w_sup_tank/EB.Q_w_sup # 상수도 에너지 대비 엑서지 비율
EB.X_c_tank/EB.E_heater # 보일러 엑서지 대비 소비 엑서지 비율
EB.X_c_mix/EB.X_l_tank # 저탕조 엑서지 손실 대비 믹싱밸브 혼합 엑서지 소비비율
EB.X_c_mix/EB.E_heater # 보일러 엑서지 대비 믹싱밸브 혼합 엑서지 소비비율
EB.X_c_mix/EB.X_w_serv # 서빙 엑서지 대비 믹싱밸브 혼합 엑서지 소비비율
EB.dV_w_sup_mix*enex.m32L/enex.s2m #m3/s
EB.dV_w_sup_tank*enex.m32L/enex.s2m
print(f"tank volume: {EB.V_tank} m3")
print(f"U_tank: {EB.U_tank} W/m2K")
print(f"Q_l_tank: {EB.Q_l_tank} W")
enex.print_balance(EB.exergy_balance,1)

tank volume: 0.1005309649148734 m3
U_tank: 0.4547668559701977 W/m2K
Q_l_tank: 27.286011358211862 W


HOT WATER TANK EXERGY BALANCE:====================

IN ENTRIES:
E_heater: 2740.4 [W]
X_w_sup_tank: 9.7 [W]

OUT ENTRIES:
X_w_tank: 312.6 [W]
X_l_tank: 4.9 [W]

CON ENTRIES:
X_c_tank: 2432.7 [W]


MIXING VALVE EXERGY BALANCE:======================

IN ENTRIES:
X_w_tank: 312.6 [W]
X_w_sup_mix: 4.2 [W]

OUT ENTRIES:
X_w_serv: 259.2 [W]

CON ENTRIES:
X_c_mix: 57.5 [W]


## GB

In [ ]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')

GB       = enex.GasBoiler()
GB.r0    = 0.12
GB.H     = 0.45
GB.x_ins = 0.10
GB.dV_w_tap = MWF
GB.system_update()

# ----------------------------
# 데이터 정의 (Energy, Entropy, Exergy 각각 Tank + Mixing 포함)
# ----------------------------
# 데이터 정의 및 딕셔너리 구성
Energy = {
    "components": [
        {"value": GB.Q_w_sup, "sign": 1, "label": 'Supply water\nto chamber'},
        {"value": GB.E_NG, "sign": 1, "label": 'Gas input\nto chamber'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": GB.Q_exh, "sign": -1, "label": 'Exhaust gas\nheat loss'},
        {"value": 0, "sign":  1, "label": ''},
        {"value": GB.Q_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": GB.Q_w_sup_mix, "sign":  1, "label": 'Supply water\nto mixer'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": GB.Q_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax"   : 4000,
    "yint"   : 1000
}
    
Entropy = {
    "components": [
        {"value": GB.S_w_sup, "sign": 1, "label": 'Supply water\nto chamber'},
        {"value": GB.S_NG, "sign": 1, "label": 'Gas input\nto chamber'},
        {"value": 0, "sign": 1, "label": 'Generation\nin chamber'},
        {"value": GB.S_g_tank, "sign": 1, "label": 'Consumption\nin tank'},
        {"value": 0, "sign": 1, "label": 'Generation\nin tank'},
        {"value": GB.S_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": GB.S_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixer'},
        {"value": GB.S_g_mix, "sign": 1, "label": 'Generation\n in mixer'},
        {"value": GB.S_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.amber:',
    "ylabel": 'Entropy [W/K]',
    "ymax"   : 16,
    "yint"   : 4
}
    
Exergy = {
    "components": [
        {"value": GB.X_w_sup, "sign": 1, "label": 'Supply water\nto chamber'},
        {"value": GB.X_NG, "sign": 1, "label": 'Gas input\nto chamber'},
        {"value": GB.X_c_comb, "sign": -1, "label": 'Consumption\nin chamber'},
        {"value": GB.X_exh, "sign": -1, "label": 'Exhaust gas\nheat loss'},
        {"value": GB.X_c_tank, "sign": -1, "label": 'Consumption\nin tank'},
        {"value": GB.X_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": GB.X_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixer'},
        {"value": GB.X_c_mix, "sign": -1, "label": 'Consumption\nin mixer'},
        {"value": GB.X_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.purple:',
    "ylabel":  'Exergy [W]',
    "ymax"   : 3000,
    "yint"   : 1000
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 3
ncols = 1

fig, axs = plt.subplots(2, 1, figsize=(dm.cm2in(14), dm.cm2in(8)), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.05  
    bm = DL * 0.1  
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(x_pos, signed, width=bw, bottom=bottoms,
                  color=color + '500', edgecolor=color + '800', linewidth=0.5)
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        va = 'bottom' if h >= 0 else 'top'
        ax.text(bar.get_x() + bw/2,
                bottoms[j] + h + (offset if h >= 0 else -offset), '' if h == 0 else f'{h:.0f}',
                ha='center', va=va, fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-2))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(min(0, min(value)*1.1), data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(1), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.065, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/GB_waterfall.svg')
dm.util.save_and_show(fig)


Mean water flow rate: 1.11 L/min


In [61]:
print(f"tank volume: {GB.V_tank} m3")
print(f"U_tank: {GB.U_tank} W/m2K")
print(f"T_w_comb_out: {enex.K2C(GB.T_w_comb)} K")
print(f"Q_l_tank: {GB.Q_l_tank} W")
enex.print_balance(GB.exergy_balance,0)

tank volume: 0.02035752039526186 m3
U_tank: 0.17303574242764017 W/m2K
T_w_comb_out: 60.191330218232736 K
Q_l_tank: 10.38214454565841 W


COMBUSTION CHAMBER EXERGY BALANCE:================

IN ENTRIES:
X_NG: 2814.0 [W]
X_w_sup: 10.0 [W]

OUT ENTRIES:
X_w_comb_out: 314.0 [W]
X_exh: 62.0 [W]

CON ENTRIES:
X_c_comb: 2448.0 [W]


HOT WATER TANK EXERGY BALANCE:====================

IN ENTRIES:
X_w_comb_out: 314.0 [W]

OUT ENTRIES:
X_w_tank: 313.0 [W]
X_l_tank: 2.0 [W]

CON ENTRIES:
X_c_tank: 0.0 [W]


MIXING VALVE EXERGY BALANCE:======================

IN ENTRIES:
X_w_tank: 313.0 [W]
X_w_sup_mix: 4.0 [W]

OUT ENTRIES:
X_w_serv: 259.0 [W]

CON ENTRIES:
X_c_mix: 57.0 [W]


## HPB

In [54]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')

HPB       = enex.HeatPumpBoiler()
HPB.r0    = 0.2
HPB.H     = 0.8
HPB.x_ins = 0.10
HPB.dV_w_tap = MWF
HPB.system_update()


Energy = {
    "components": [
        {"value": 0, "sign": 1, "label": ''},
        {"value": HPB.Q_r_ext, "sign": 1, "label": 'Heat transfer\nfrom air to\next unit ref'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": 0, "sign": -1, "label": ''},
        {"value": 0, "sign":  1, "label": ''},
        {"value": HPB.E_cmp, "sign": 1, "label": 'Elec input\nto comp'},
        {"value": 0, "sign": -1, "label": ''},
        {"value": 0, "sign": 1, "label": ''},
        {"value": HPB.Q_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": HPB.Q_w_sup_tank, "sign":  1, "label": 'Supply water\nto tank'},
        {"value": HPB.Q_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax"   : 4000,
    "yint"   : 1000
}
    
Entropy = {
    "components": [
        {"value": HPB.S_fan, "sign": 1, "label": 'Elec input\nto fan'},
        {"value": HPB.S_r_ext, "sign": 1, "label": 'Heat transfer\nfrom air to\next unit ref'},
        {"value": HPB.S_a_ext_in, "sign": 1, "label": 'Inlet air\nto ext unit'},
        {"value": HPB.S_g_ext, "sign": -1, "label": 'Generation\next unit'},
        {"value": HPB.S_a_ext_out, "sign":  1, "label": 'Exhaust air\nfrom ext unit'},
        {"value": HPB.S_cmp, "sign": -1, "label": 'Elec input\nto comp'},
        {"value": HPB.S_g_r, "sign": -1, "label": 'Generation\nin ref loop'},
        {"value": HPB.S_r_ext, "sign": -1, "label": 'Ext unit ref\nto air'},
        {"value": HPB.S_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": HPB.S_g_tank, "sign": 1, "label": 'Generation\nin tank'},
        {"value": HPB.S_w_sup_tank, "sign":  1, "label": 'Supply water\nto tank'},
        {"value": HPB.S_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.amber:',
    "ylabel": 'Entropy [W/K]',
    "ymax"   : 16,
    "yint"   : 4
}
    
Exergy = {
    "components": [
        {"value": HPB.X_fan, "sign": 1, "label": 'Elec input\nto fan'},
        {"value": HPB.X_r_ext, "sign": 1, "label": 'Heat transfer\nfrom air to\next unit ref'},
        {"value": HPB.X_a_ext_in, "sign": 1, "label": 'Inlet air\nto ext unit'},
        {"value": HPB.X_c_ext, "sign": -1, "label": 'Consumption\next unit'},
        {"value": HPB.X_a_ext_out, "sign":  -1, "label": 'Exhaust air\nfrom ext unit'},
        {"value": HPB.X_cmp, "sign": 1, "label": 'Elec input\nto comp'},
        {"value": HPB.X_c_r, "sign": -1, "label": 'Consumption\nin ref loop'},
        {"value": HPB.X_r_ext, "sign": -1, "label": 'Ext unit ref\nto air'},
        {"value": HPB.X_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": HPB.X_c_tank, "sign": -1, "label": 'Consumption\nin tank'},
        {"value": HPB.X_w_sup_tank, "sign":  1, "label": 'Supply water\nto tank'},
        {"value": HPB.X_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.purple:',
    "ylabel": 'Exergy [W]',
    "ymax"   : 4000,
    "yint"   : 1000
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 2
ncols = 1

fig, axs = plt.subplots(2, 1, figsize=(dm.cm2in(18), dm.cm2in(8)), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.035  
    bm = DL * 0.05  
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(x_pos, signed, width=bw, bottom=bottoms,
                  color=color + '500', edgecolor=color + '800', linewidth=0.5)
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
        
        va = 'bottom' if h >= 0 else 'top'
        text_loc = bottoms[j] + h + (offset if h >= 0 else -offset)
        if text_loc < 0:
            va = 'bottom'
            text_loc = bottoms[j] - h
        text_val = '' if h == 0 else f'{h:.0f}'
        
        ax.text(bar.get_x() + bw/2,
                text_loc, text_val,
                ha='center', va=va, fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-2))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(0, data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(1), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.065, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/GB_waterfall.svg')
dm.util.save_and_show(fig)

Mean water flow rate: 1.11 L/min


## SHW

In [4]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
# print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')   
SHW = enex.SolarHotWater()
SHW.dV_w_serv = MWF
SHW.I_DN = 500
SHW.I_dH = 200
SHW.A_stp = 4
SHW.system_update()
enex.print_balance(SHW.exergy_balance,0)
print(f"{SHW.S_dH/SHW.S_DN}")



SOLAR THERMAL PANEL EXERGY BALANCE:===============

IN ENTRIES:
X_sol: 2581.0 [W]
X_w_sup: 10.0 [W]

CON ENTRIES:
X_c_stp: 2119.0 [W]

OUT ENTRIES:
X_w_stp_out: 91.0 [W]
X_l: 381.0 [W]


COMBUSTION CHAMBER EXERGY BALANCE:================

IN ENTRIES:
X_w_stp_out: 91.0 [W]
X_NG: 1606.0 [W]

CON ENTRIES:
X_c_comb: 1349.0 [W]

OUT ENTRIES:
X_exh: 35.0 [W]
X_w_comb: 313.0 [W]


MIXING VALVE EXERGY BALANCE:======================

IN ENTRIES:
X_w_comb: 313.0 [W]
X_w_sup_mix: 4.0 [W]

CON ENTRIES:
X_c_mix: 57.0 [W]

OUT ENTRIES:
X_w_serv: 259.0 [W]
1.3284342138002634


In [2]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
# print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')    
SHW = enex.SolarHotWater()
SHW.dV_w_serv = MWF
SHW.I_DN = 500
SHW.I_dH = 200
SHW.x_ins = 0.05
SHW.A_stp = 4
SHW.system_update()

# ----------------------------
# 데이터 정의 (Energy, Entropy, Exergy 각각 Tank + Mixing 포함)
# ----------------------------
# 데이터 정의 및 딕셔너리 구성


Energy = {
    "components": [
        {"value": SHW.Q_w_sup, "sign": 1, "label": 'Supply water\nto solar panel'},
        {"value": SHW.Q_sol, "sign": 1, "label": 'Solar radiation\nto solar panel'},
        {"value": 0, "sign":  1, "label": ''},
        {"value": SHW.Q_l, "sign": -1, "label": 'Heat loss\nfrom solar panel'},
        {"value": SHW.E_NG, "sign": 1, "label": 'Gas input\nto chamber'},
        {"value": 0, "sign": -1, "label": ''},
        {"value": SHW.Q_exh, "sign": -1, "label": 'Exhaust gas\nheat loss'},
        {"value": SHW.Q_w_sup_mix, "sign":  1, "label": 'Supply water\nto mixing valve'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": SHW.Q_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax"   : 3600,
    "yint"   : 900
}
    
Exergy = {
    "components": [
        {"value": SHW.X_w_sup, "sign": 1, "label": 'Supply water\nto solar panel'},
        {"value": SHW.X_sol, "sign": 1, "label": 'Solar radiation\nto solar panel'},
        {"value": SHW.X_c_stp, "sign": -1, "label": 'Consumption\nin solar panel'},
        {"value": SHW.X_l, "sign": -1, "label": 'Heat loss\nfrom solar panel'},
        {"value": SHW.X_NG, "sign": 1, "label": 'Gas input\nto chamber'},
        {"value": SHW.X_c_comb, "sign": -1, "label": 'Consumption\nin chamber'},
        {"value": SHW.X_exh, "sign": -1, "label": 'Exhaust gas\nheat loss'},
        {"value": SHW.X_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixing valve'},
        {"value": SHW.X_c_mix, "sign": -1, "label": 'Consumption\nin mixing valve'},
        {"value": SHW.X_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.purple:',
    "ylabel":  'Exergy [W]',
    "ymax"   : 3600,
    "yint"   : 900
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 2
ncols = 1

fig, axs = plt.subplots(nrows, ncols, figsize=(dm.cm2in(20), dm.cm2in(7)), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.03  
    bm = DL * 0.04 
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-1))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(min(0, min(value)*1.1), data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(0), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.05, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/SHW_waterfall.svg')
plt.savefig('figure/SHW_waterfall.png',dpi=600)
dm.util.save_and_show(fig)

## EH

In [11]:
EH = enex.ElectricHeater()
EH.T0 = 0
EH.T_mr = 10
EH.D_p = 0.005 # 
EH.E_heater = 1000 # [W]
EH.W_p = 1 # [m3/s]
EH.dt = 10
EH.system_update()

Energy = {
    "components": [
        {"value": EH.E_heater, "sign": 1, "label": 'Electricity\ninput'},
        {"value": EH.Q_rad_mr, "sign": 1, "label": 'Radiation\n(main room)'},
        {"value": EH.Q_rad_ps, "sign": -1, "label": 'Radiation\n(peripheral)'},
        {"value": EH.Q_conv_ps, "sign": -1, "label": 'Convection\n(peripheral)'},
    ],
    "color": 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax": 1200,
    "yint": 300
}

Exergy = {
    "components": [
        {"value": EH.X_heater, "sign": 1, "label": 'Electricity\ninput'},
        {"value": EH.X_rad_mr, "sign": 1, "label": 'Radiation\n(main room)'},
        {"value": EH.X_rad_ps, "sign": -1, "label": 'Radiation\n(peripheral)'},
        {"value": EH.X_conv_ps, "sign": -1, "label": 'Convection\n(peripheral)'},
    ],
    "color": 'tw.purple:',
    "ylabel": 'Exergy [W]',
    "ymax": 1200,
    "yint": 300
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 2
ncols = 1

fig, axs = plt.subplots(nrows, ncols, figsize=(dm.cm2in(20), dm.cm2in(8)), dpi=200)

for i, data in enumerate(data_list):
    ax = axs[i]
    
    # Components
    components = data["components"]
    value = np.array([comp["value"] for comp in components])
    sign = np.array([comp["sign"] for comp in components])
    labels = [comp["label"] for comp in components]
    signed = value * sign
    color = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.03  
    bm = DL * 0.04 
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1] = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
                ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-1))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize=dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(min(0, min(value)*1.1), data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(0), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.05, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/EH_waterfall.svg')
plt.savefig('figure/EH_waterfall.png', dpi=600)
dm.util.save_and_show(fig)

# Test

## Sobol

In [10]:
# Sobol sensitivity analysis
Si = sobol.analyze(problem, np.array(ex_eff_EB), print_to_console=False)

# Visualize first-order sensitivity indices
fig, ax = plt.subplots(figsize=(dm.cm2in(10), dm.cm2in(5)), dpi=600)

# Bar plot for first-order sensitivity indices
ax.bar(problem['names'], Si['S1'], color='tw.indigo:700', alpha=0.7)

# Add labels and title
ax.set_ylabel('First-order Sensitivity Index', fontsize=dm.fs(0), fontweight=300)
ax.set_xlabel('Input Variables', fontsize=dm.fs(0), fontweight=300)
ax.set_title('Sobol Sensitivity Analysis', fontsize=dm.fs(1), fontweight=400)

# Layout adjustments
dm.simple_layout(fig, bbox=(0, 1, 0.01, 1), verbose=False)

# Save and show the figure
fig_name = 'sobol_sensitivity_EB'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)
dm.util.save_and_show(fig)
plt.close()

NameError: name 'problem' is not defined

## EH transient

In [4]:
EH = enex.ElectricHeater()
EH.T0 = 0
EH.T_mr = 10
EH.D_p = 0.005 # 
EH.E_heater = 1000 # [W]
EH.W_p = 1 # [m3/s]
EH.dt = 10
EH.system_update()

time = np.array(EH.time) / 3600 # convert to hours
print(EH.V_p)

# Energy balance check
# EnergyBalance1 = np.array(EH.Q_cond_list) + np.array(EH.Q_rad_mr_list) - np.array(EH.Q_rad_ps_list) - np.array(EH.Q_conv_ps_list)
# EnergyBalance2 = np.array(EH.E_heater_list) - np.array(EH.Q_stored_list) - np.array(EH.Q_cond_list)

# plt.plot(time, EnergyBalance1, label="Energy balance 1", color = "tw.amber:500")
# plt.plot(time, EnergyBalance2, label="Energy balance 2", color = "tw.blue:600")
# plt.legend()

#%% 
# Tempearature
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
ax.plot(time, enex.K2C(np.array(EH.T_p_list)), label="$T_{p}$", color = "tw.amber:500",linestyle='-')
ax.plot(time, enex.K2C(np.array(EH.T_ps_list)), label="$T_{ps}$", color = "tw.lime:600",linestyle='-.')
ax.set_ylim(0, 100*1.01)
ax.set_xlabel("Time [hours]", fontsize=dm.fs(0))
ax.set_ylabel("Temperature [°C]", fontsize=dm.fs(0))

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='upper left', fontsize=dm.fs(-2), frameon=False, ncol=3)

dm.simple_layout(fig=fig, bbox =(0, 1, 0, 1), margins=(0.1, 0.1, 0.1, 0.1), verbose = False)
plt.savefig("figure/EH_temp.png", dpi = 600)
plt.savefig("figure/EH_temp.svg", transparent=True)
dm.save_and_show(fig)

#%% 
# Energy balance (surface)
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
ax.plot(time, np.array(EH.Q_cond_list), label="$Q_{cond}$", color = "tw.lime:600",)
ax.plot(time, np.array(EH.Q_rad_mr_list), label="$Q_{rad,mr}$", color = "tw.amber:500",)
ax.plot(time, np.array(EH.Q_rad_ps_list), label="$Q_{rad,ps}$", color = "tw.blue:600",)
ax.plot(time, np.array(EH.Q_conv_ps_list), label="$Q_{conv,ps}$", color = "tw.purple:600",) 
ax.set_ylim(0, 1000*1.01)

ax.set_xlabel("Time [hours]", fontsize=dm.fs(0))
ax.set_ylabel("Energy [W]", fontsize=dm.fs(0))

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='upper left', fontsize=dm.fs(-2), frameon=False, ncol=4, handlelength=1.5)

dm.simple_layout(fig=fig, bbox =(0, 1, 0, 1), margins=(0.1, 0.1, 0.1, 0.1), verbose = False)
plt.savefig("figure/EH_energy balance(surf).png", dpi = 600)
plt.savefig("figure/EH_energy balance(surf).svg", transparent=True)
dm.save_and_show(fig)
#%% 
# plotting
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)

# Energy balance (body)
ax.plot(time, np.array(EH.E_heater_list), label="$E_{heater}$", color = "tw.amber:500",)
ax.plot(time, np.array(EH.Q_cond_list), label="$Q_{cond}$", color = "tw.lime:600")
ax.plot(time, np.array(EH.Q_stored_list), label="$Q_{stored}$", color = "tw.purple:600") 
ax.set_ylim(0, 1000*1.01)
ax.set_xlabel("Time [hours]", fontsize=dm.fs(0))
ax.set_ylabel("Energy [W]", fontsize=dm.fs(0))

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='lower right', fontsize=dm.fs(-2), frameon=False, ncol=3)

dm.simple_layout(fig=fig, bbox =(0, 1, 0, 1), margins=(0.1, 0.1, 0.1, 0.1), verbose = False)
plt.savefig("figure/EH_energy balance(body).png", dpi = 600)
plt.savefig("figure/EH_energy balance(body).svg", transparent=True)
dm.save_and_show(fig)


# plotting
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
# Exergy balance (surf)
ax.plot(time, np.array(EH.X_heater_list), label="$X_{heater}$", color = "tw.amber:500",)
ax.plot(time, np.array(EH.X_cond_list), label="$X_{cond}$", color = "tw.lime:600")
ax.plot(time, np.array(EH.X_cond_list)+np.array(EH.X_stored_list), label="$X_{cond}+X_{stored}$", color = "tw.purple:600") 
ax.set_ylim(0, 1000*1.01)
ax.set_xlabel("Time [hours]", fontsize=dm.fs(0))
ax.set_ylabel("Exergy [W]", fontsize=dm.fs(0))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='upper right', fontsize=dm.fs(-2), frameon=False, ncol=4,)

dm.simple_layout(fig=fig, bbox =(0, 1, 0, 1), verbose = False)
plt.savefig("figure/EH_exergy balance(body).png", dpi = 600)
plt.savefig("figure/EH_exergy balance(body).svg", transparent=True)
dm.save_and_show(fig)


# plotting
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
# Exergy balance (surf)
ax.plot(time, np.array(EH.X_cond_list), label="$X_{cond}$", color = "tw.lime:600",)
ax.plot(time, np.array(EH.X_cond_list)+np.array(EH.X_rad_mr_list), label="$X_{cond}+X_{rad,mr}$", color = "tw.amber:500",linestyle='-.')
ax.plot(time, np.array(EH.X_rad_ps_list), label="$X_{rad,ps}$", color = "tw.blue:600",)
ax.plot(time, np.array(EH.X_rad_ps_list)+np.array(EH.X_conv_ps_list), label="$X_{rad,ps}+X_{conv,ps}$", color = "tw.purple:600",) 
ax.set_ylim(0, 300*1.01)
ax.set_xlabel("Time [hours]", fontsize=dm.fs(0))
ax.set_ylabel("Exergy [W]", fontsize=dm.fs(0))

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='upper right', fontsize=dm.fs(-2), frameon=False, ncol=4,)

dm.simple_layout(fig=fig, bbox =(0, 1, 0, 1), verbose = False)
plt.savefig("figure/EH_exergy balance(surf).png", dpi = 600)
plt.savefig("figure/EH_exergy balance(surf).svg", transparent=True)
dm.save_and_show(fig)

0.0025


In [ ]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')

I_DN_range = np.arange(200, 800, 1)

T_w_stp_out_list = []
T_sp_list = []

Q_sol_list = []
Q_w_sup_list = []
Q_w_stp_out_list = []
Q_l_list = []
E_NG_list = []
Q_exh_list = []
Q_w_comb_list = []
Q_w_sup_mix_list = []
Q_w_serv_list = []

X_sol_list = []
X_w_sup_list = []
X_w_stp_out_list = []
X_l_list = []
X_NG_list = []
X_exh_list = []
X_w_comb_list = []
X_w_sup_mix_list = []
X_w_serv_list = []

for DN_idx in I_DN_range:
    SHW = enex.SolarHotWater()
    SHW.dV_w_serv = MWF
    SHW.I_DN = DN_idx
    SHW.system_update()
    
    T_w_stp_out_list.append(SHW.T_w_stp_out)
    T_sp_list.append(SHW.T_sp)
    
    Q_sol_list.append(SHW.Q_sol)
    Q_w_sup_list.append(SHW.Q_w_sup)
    Q_w_stp_out_list.append(SHW.Q_w_stp_out)
    Q_l_list.append(SHW.Q_l)
    E_NG_list.append(SHW.E_NG)
    Q_exh_list.append(SHW.Q_exh)
    Q_w_comb_list.append(SHW.Q_w_comb)
    Q_w_sup_mix_list.append(SHW.Q_w_sup_mix)
    Q_w_serv_list.append(SHW.Q_w_serv)
    
    X_sol_list.append(SHW.X_sol)
    X_w_sup_list.append(SHW.X_w_sup)
    X_w_stp_out_list.append(SHW.X_w_stp_out)
    X_l_list.append(SHW.X_l)
    X_NG_list.append(SHW.X_NG)
    X_exh_list.append(SHW.X_exh)
    X_w_comb_list.append(SHW.X_w_comb)
    X_w_sup_mix_list.append(SHW.X_w_sup_mix)
    X_w_serv_list.append(SHW.X_w_serv)
    

Mean water flow rate: 1.11 L/min


## SHW 

In [ ]:
#%% Temperature
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
ax.plot(I_DN_range, enex.K2C(np.array(T_w_stp_out_list)), label="$T_{w,stp,out}$", color="tw.lime:600", linestyle='-')
ax.plot(I_DN_range, enex.K2C(np.array(T_sp_list)), label="$T_{sp}$", color="tw.amber:500", linestyle='-')

ax.set_xlabel("Direct solar irradiation $I_{NG}$ [W/m$^2$]", fontsize=dm.fs(0))
ax.set_ylabel("Temperature [°C]", fontsize=dm.fs(0))
ax.set_xticks(np.arange(200, 1000, 200))

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))
ax.set_ylim(0, 100*1.01)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='best', fontsize=dm.fs(-2), frameon=False, ncol=4)

dm.simple_layout(fig=fig, bbox=(0, 1, 0, 1), margins=(0.1, 0.1, 0.1, 0.1), verbose=False)
plt.savefig("figure/SHW_T_w_stp_out.svg", transparent=True)
dm.save_and_show(fig)


#%% Energy balance - solar thermal panel
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
ax.plot(I_DN_range, Q_sol_list, label="$Q_{sol}$", color="tw.amber:500", linestyle='-')
ax.plot(I_DN_range, Q_w_sup_list, label="$Q_{w,sup}$", color="tw.lime:600", linestyle='-')
ax.plot(I_DN_range, Q_w_stp_out_list, label="$Q_{w,stp,out}$", color="tw.blue:600", linestyle='-')
ax.plot(I_DN_range, Q_l_list, label="$Q_{l}$", color="tw.purple:600", linestyle='-')
ax.set_ylim(0, 2000*1.01)


ax.set_xlabel("Direct solar irradiation $I_{NG}$ [W/m$^2$]", fontsize=dm.fs(0))
ax.set_ylabel("Energy [W]", fontsize=dm.fs(0))
ax.set_xticks(np.arange(200, 1000, 200))

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='best', fontsize=dm.fs(-2), frameon=False, ncol=4)

dm.simple_layout(fig=fig, bbox=(0, 1, 0, 1), margins=(0.1, 0.1, 0.1, 0.1), verbose=False)
plt.savefig("figure/SHW_energy_balance_collector.svg", transparent=True)
dm.save_and_show(fig)


#%% Energy balance - combustion chamber
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
ax.plot(I_DN_range, Q_w_stp_out_list, label="$Q_{w,stp,out}$", color="tw.amber:500", linestyle='-')
ax.plot(I_DN_range, E_NG_list, label="$E_{NG}$", color="tw.lime:600", linestyle='-')
ax.plot(I_DN_range, Q_w_comb_list, label="$Q_{w,comb}$", color="tw.blue:600", linestyle='-')
ax.plot(I_DN_range, Q_exh_list, label="$Q_{exh}$", color="tw.purple:600", linestyle='-')

ax.set_xlabel("Direct solar irradiation $I_{NG}$ [W/m$^2$]", fontsize=dm.fs(0))
ax.set_ylabel("Energy [W]", fontsize=dm.fs(0))
ax.set_xticks(np.arange(200, 1000, 200))
ax.set_ylim(0, 4000*1.01)

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='best', fontsize=dm.fs(-2), frameon=False, ncol=4)

dm.simple_layout(fig=fig, bbox=(0, 1, 0, 1), margins=(0.1, 0.1, 0.1, 0.1), verbose=False)
plt.savefig("figure/SHW_energy_balance_combustion.svg", transparent=True)
dm.save_and_show(fig)


#%% Energy balance - mixing valve
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
ax.plot(I_DN_range, Q_w_comb_list, label="$Q_{w,comb}$", color="tw.amber:500", linestyle='-')
ax.plot(I_DN_range, Q_w_sup_mix_list, label="$Q_{w,sup,mix}$", color="tw.lime:600", linestyle='-')
ax.plot(I_DN_range, Q_w_serv_list, label="$Q_{w,serv}$", color="tw.blue:600", linestyle='-')

ax.set_xlabel("Direct solar irradiation $I_{NG}$ [W/m$^2$]", fontsize=dm.fs(0))
ax.set_ylabel("Energy [W]", fontsize=dm.fs(0))
ax.set_xticks(np.arange(200, 1000, 200))
ax.set_ylim(0, 4000*1.01)

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='best', fontsize=dm.fs(-2), frameon=False, ncol=4)

dm.simple_layout(fig=fig, bbox=(0, 1, 0, 1), margins=(0.1, 0.1, 0.1, 0.1), verbose=False)
plt.savefig("figure/SHW_energy_balance_mixing.svg", transparent=True)
dm.save_and_show(fig)


In [ ]:
#%% exergy balance - solar thermal panel
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
ax.plot(I_DN_range, np.array(X_sol_list), label="$X_{sol}$", color="tw.amber:500", linestyle='-')
ax.plot(I_DN_range, np.array(X_sol_list)+np.array(X_w_sup_list), label="$X_{sol}+X_{w,sup}$", color="tw.lime:600", linestyle='-.')
ax.plot(I_DN_range, np.array(X_w_stp_out_list), label="$X_{w,stp,out}$", color="tw.blue:600", linestyle='-')
ax.plot(I_DN_range, np.array(X_w_stp_out_list)+np.array(X_l_list), label="$X_{w,stp,out}+X_{l}$", color="tw.purple:600", linestyle='-')
ax.set_yscale('log')
ax.set_ylim(1, 2000*1.01)


ax.set_xlabel("Direct solar irradiation $I_{NG}$ [W/m$^2$]", fontsize=dm.fs(0))
ax.set_ylabel("Exergy [W]", fontsize=dm.fs(0))
ax.set_xticks(np.arange(200, 1000, 200))

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='best', fontsize=dm.fs(-2), frameon=False, ncol=4)

dm.simple_layout(fig=fig, bbox=(0, 1, 0, 1), margins=(0.1, 0.1, 0.1, 0.1), verbose=False)
plt.savefig("figure/SHW_exergy_balance_collector.svg", transparent=True)
dm.save_and_show(fig)


#%% exergy balance - combustion chamber
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
ax.plot(I_DN_range, np.array(X_w_stp_out_list), label="$X_{w,stp,out}$", color="tw.amber:500", linestyle='-')
ax.plot(I_DN_range, np.array(X_w_stp_out_list)+np.array(E_NG_list), label="$X_{w,stp,out}+X_{NG}$", color="tw.lime:600", linestyle='-')
ax.plot(I_DN_range, np.array(X_w_comb_list), label="$X_{w,comb}$", color="tw.blue:600", linestyle='-')
ax.plot(I_DN_range, np.array(X_w_comb_list)+np.array(X_exh_list), label="$X_{w,comb}+X_{exh}$", color="tw.purple:600", linestyle='-')

ax.set_xlabel("Direct solar irradiation $I_{NG}$ [W/m$^2$]", fontsize=dm.fs(0))
ax.set_ylabel("Exergy [W]", fontsize=dm.fs(0))
ax.set_xticks(np.arange(200, 1000, 200))
ax.set_yscale('log')
ax.set_ylim(1, 10000*1.01)

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='best', fontsize=dm.fs(-2), frameon=False, ncol=4)

dm.simple_layout(fig=fig, bbox=(0, 1, 0, 1), margins=(0.1, 0.1, 0.1, 0.1), verbose=False)
plt.savefig("figure/SHW_exergy_balance_combustion.svg", transparent=True)
dm.save_and_show(fig)


#%% exergy balance - mixing valve
fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(10), dm.cm2in(4)), sharex=True)
ax.plot(I_DN_range, X_w_comb_list, label="$X_{w,comb}$", color="tw.amber:500", linestyle='-')
ax.plot(I_DN_range, np.array(X_w_comb_list)+np.array(X_w_sup_mix_list), label="$X_{w,comb}+X_{w,sup,mix}$", color="tw.lime:600", linestyle='-.')
ax.plot(I_DN_range, X_w_serv_list, label="$X_{w,serv}$", color="tw.blue:600", linestyle='-')

ax.set_xlabel("Direct solar irradiation $I_{NG}$ [W/m$^2$]", fontsize=dm.fs(0))
ax.set_ylabel("Exergy [W]", fontsize=dm.fs(0))
ax.set_xticks(np.arange(200, 1000, 200))
ax.set_ylim(1, 400*1.01)

ax.tick_params(axis='both', which='major', labelsize=dm.fs(-1))
ax.tick_params(axis='both', which='minor', labelsize=dm.fs(-1))
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='best', fontsize=dm.fs(-2), frameon=False, ncol=4)

dm.simple_layout(fig=fig, bbox=(0, 1, 0, 1), margins=(0.1, 0.1, 0.1, 0.1), verbose=False)
plt.savefig("figure/SHW_exergy_balance_mixing.svg", transparent=True)
dm.save_and_show(fig)


## EH

In [4]:
EH = enex.ElectricHeater()
EH.T0 = 0
EH.T_mr = 10
EH.D = 0.005 # 
EH.E_heater = 1000 # [W]
EH.W = 1 # [m3/s]
EH.dt = 10
EH.system_update()

enex.print_balance(EH.exergy_balance)



HEATER BODY EXERGY BALANCE:=======================

IN ENTRIES:
X_heater: 1000.0 [W]

CON ENTRIES:
X_c_hb: 760.93 [W]

OUT ENTRIES:
X_st: 0.0 [W]
X_cond: 239.07 [W]


HEATER SURFACE EXERGY BALANCE:====================

IN ENTRIES:
X_cond: 239.07 [W]
X_rad_rs: 0.89 [W]

CON ENTRIES:
X_c_hs: 47.39 [W]

OUT ENTRIES:
X_conv: 101.08 [W]
X_rad_hs: 91.49 [W]


In [ ]:
T0 = enex.C2K(0)
Tw = enex.C2K(40)
EX = enex.Exergy()

c_w  = 4186 # [J/(kg*K)]
rho_w = 1000 # [kg/m3]

Q = c_w * rho_w * (Tw - T0) # [J/kgs]
c_eff = (1-T0/Tw)

X = c_eff*Q

